This notebook creates interpolations of potential energy as a functions of radial distance from a wire for Rydberg atoms. This the final version of our model and is based upon Mathematica code created by Anne Goodsell. The version in "Original Workflow" contains multiple visuals to better demonstrate what each cell is doing. These have been removed from this version to make the notebook as concise as possible. 

FFW | 10/19/2023

In [10]:
#Upload basic packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import math
import os
from scipy import constants
import matplotlib.patches as mpatches
import pickle

#### Loading in Stark Effect Data:

In [11]:
#Determing server directory
current_directory = os.getcwd()
#print(current_directory)

In [12]:
#Initialize path to main Stark effect Folder
fpath0 = '/home/anaconda3/Wimberly_PHYS704_F23/Stark Map CSV files/'
file = 'StarkMapPlotList_FieldVcm_Energy_n35_{}.csv'

# Generate list of all energy sublevels
sublevels = list(range(284, 347))

starkmap_Vcm_cm = {}
starkmap_Vm_J = {}

for sublevel in sublevels:
    filename = (fpath0 + file).format(sublevel)
    starkmap_Vcm_cm[sublevel] = pd.read_csv(filename, index_col=None, header=None, skiprows = 1)
    temp_df = starkmap_Vcm_cm[sublevel].copy()            #Copy the df
    temp_df[0] *= 1e2                                  #Convert V/cm to V/m
    temp_df[1] *= 1e2                                  #Convert cm^-1 to m^-1
    temp_df[1] *= (constants.h*constants.c)            #Convert energy from wavenumber (m^-1) to joules
    starkmap_Vm_J[sublevel] = temp_df
    starkmap_Vm_J[sublevel].columns = [ r'E Field $(V/m)$', 'U $(J)$']         #Label corrected units

FileNotFoundError: [Errno 2] No such file or directory: '/home/anaconda3/Wimberly_PHYS704_F23/Stark Map CSV files/StarkMapPlotList_FieldVcm_Energy_n35_284.csv'

#### Initializing the wire and field

In [36]:
#Defining E(r) and r(E)
def Efield(r):
    """
    Calculate electric field strength at a given radial distance from a wire.
    
    Args:
        r (float): The radial distance from the wire in meters.
        
    Returns:
        float: The electric field strength in volts per meter (V/m) at the specified distance. """
    
    E = (1 / r) * VHere / (math.log(R0 / Rwire))  # Volts per meter
    
    return E

In [37]:
def r(Efield):
    """
    Calculate the radial distance from a wire corresponding to a given electric field strength.
    
    Args:
        Efield (float): The electric field strength in volts per meter (V/m).
        
    Returns:
        float: The radial distance from the wire in meters that corresponds to the 
        specified electric field strength. """
    
    r = (1 / Efield) * VHere / (math.log(R0 / Rwire))
    
    return r

In [38]:
#Defining some constants

R0 = 0.15  # R0 = 0.15 m = 15 cm (not super important; imaginary radius at which electric potential is zero)
Rwire = 15e-6  # Radius of the wire = 15 microns (15*10^-6 meters)
VHere = 3  #Voltage of wire creating E field

#### Creating Interpolations Functions:

In [39]:
# Relating distance and energy for a given field strength

r_of_E = {}
starkmap_r = {}

for sublevel in sublevels:
    starkmap_r[sublevel] = starkmap_Vm_J[sublevel].copy()  #Create new df
    # Calculate and add column with the radial distance
    starkmap_r[sublevel]['Radial Distance (m)'] = r(starkmap_Vm_J[sublevel]['E Field $(V/m)$'])  

#### Outputing $U(r)$ files:

In [40]:
#Use BSpline interpolation as it has built in derivative function
from scipy.interpolate import BSpline

sorted_starkmap_r = {}
energy_interpolation = {}
for sublevel in sublevels:
    sorted_starkmap_r[sublevel] = starkmap_r[sublevel].sort_values(by='Radial Distance (m)')
    energy_interpolation[sublevel] = BSpline(sorted_starkmap_r[sublevel]['Radial Distance (m)'], sorted_starkmap_r[sublevel]['U $(J)$'], k = 1) 
#K = 1 turns out to produce the best fit

In [41]:
#Write out interpolations to CSVs

# Define the directory to save the pickle files
output_dir = current_directory + '/Interpolations/U(r)/'

for sublevel in sublevels:
    
    # Define the full path of the output file
    fname = f"U(r)_{sublevel}.pkl"
    output_path = os.path.join(output_dir, fname)

    # Save the interpolation as a pickle file
    with open(output_path, 'wb') as file:
        pickle.dump(energy_interpolation[sublevel], file)

#### Outputing $\nabla U$ files:

In [42]:
#Solving for grad_U with built in derivative function

grad_U = {}
for sublevel in sublevels: 
    grad_U[sublevel] = energy_interpolation[sublevel].derivative()

In [43]:
#Saving grad U(r)

# Define the directory to save the pickle files
output_dir = current_directory + '/Interpolations/grad_U/'

for sublevel in sublevels:
    
    # Define the full path of the output file
    fname = f"grad_U_{sublevel}.pkl"
    output_path = os.path.join(output_dir, fname)

    # Save the interpolation as a pickle file
    with open(output_path, 'wb') as file:
        pickle.dump(grad_U[sublevel], file)